In [9]:
import scanpy as sc
import anndata
import muon as mu
import scparadise
import os
import torch
import pandas as pd
import warnings 
warnings.simplefilter('ignore')

In [10]:
# Load normalized integrated data
mdata = mu.read_h5mu('/mnt/c/Users/vadim/Desktop/R/PBMC_ref/CITEseq/3p/mdata_unintegrated.h5mu')

In [11]:
# Select RNA modality from MuData object (mdata)
adata = mdata.mod['rna'].copy()
del mdata

In [12]:
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/genes_for_AI.csv')
adata = adata[:, genes.genes].copy()

In [13]:
# Create list of samples to leave in reference train dataset
lst_reference = ['P1_0', 'P2_0', 'P3_0', 'P4_0', 'P5_0', 'P6_0', 'P7_0', 'P8_0']

In [14]:
# Create adata_train - 8 samples of 8 donors from unintegrated adata object (8 donors, 24 samples)
adata_train = adata[adata.obs['orig.ident'].isin(lst_reference)].copy()

In [15]:
# Balance dataset based on l3 annotation level
adata_balanced = scparadise.scnoah.balance(adata_train, 
                                           sample='orig.ident',
                                           celltype_l1='celltype_l1',
                                           celltype_l2='celltype_l2',
                                           celltype_l3='celltype_l3')

Successfully undersampled cell types: CD14 Mono, NK, CD4 T Naive, CD4 TCM, CD8 TEM, CD8 T Naive, CD16 Mono

Successfully oversampled cell types: B naive κ, CD4 TEM, gdT, CD8 TCM, cDC2, MAIT, Treg, B naive λ, B memory κ, CD4 CTL, Platelet, B int λ, B memory λ, B int κ, NK_CD56bright, pDC, HSPC, dnT, Plasmablast, NK Prolif, cDC1, ILC, CD4 T Prolif, ASDC, CD8 T Prolif


In [8]:
# Train scadam model using adata_balanced dataset
scparadise.scadam.train(adata_balanced,
                        path='/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/',
                        model_name='model_small_balanced',
                        celltype_l1='celltype_l1',
                        celltype_l2='celltype_l2',
                        celltype_l3='celltype_l3',
                        eval_metric=['accuracy'])

Successfully saved genes names for training model

Successfully saved dictionary of dataset annotations

Train dataset contains: 47808 cells, it is 90.0 % of input dataset
Test dataset contains: 5312 cells, it is 10.0 % of input dataset

Accelerator: cuda
Start training
epoch 0  | loss: 2.32723 | train_accuracy: 0.51008 | valid_accuracy: 0.51123 |  0:00:02s
epoch 1  | loss: 1.20643 | train_accuracy: 0.70633 | valid_accuracy: 0.70325 |  0:00:04s
epoch 2  | loss: 0.84622 | train_accuracy: 0.79572 | valid_accuracy: 0.79374 |  0:00:06s
epoch 3  | loss: 0.68229 | train_accuracy: 0.85555 | valid_accuracy: 0.85241 |  0:00:08s
epoch 4  | loss: 0.57651 | train_accuracy: 0.89244 | valid_accuracy: 0.88924 |  0:00:10s
epoch 5  | loss: 0.5237  | train_accuracy: 0.90522 | valid_accuracy: 0.9023  |  0:00:12s
epoch 6  | loss: 0.47854 | train_accuracy: 0.91555 | valid_accuracy: 0.91039 |  0:00:14s
epoch 7  | loss: 0.45391 | train_accuracy: 0.92091 | valid_accuracy: 0.91767 |  0:00:16s
epoch 8  | loss: 

In [22]:
# Create lists with paired test samples
lst_test = ['P1_3_P3_3', 'P1_7_P8_3', 'P2_3_P4_7', 'P2_7_P6_3', 'P3_7_P7_3', 'P4_3_P7_7', 'P5_3_P8_7', 'P5_7_P6_7']

In [10]:
for folder in lst_test:
    adata_test = adata[adata.obs['orig.ident'].isin([folder[0:4], folder[5:9]])].copy()
    # Predict annotation levels using pretrained scadam model
    adata_test = scparadise.scadam.predict(adata_test, 
                                           path_model = '/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/model_small_balanced/')
    # Create and save classification report of annotation levels
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l1', 
                                          pred_celltype='pred_celltype_l1', 
                                          report_name = 'report_test_model_scadam_635_AdamW_celltype_l1.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l2', 
                                          pred_celltype='pred_celltype_l2', 
                                          report_name = 'report_test_model_scadam_635_AdamW_celltype_l2.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l3', 
                                          pred_celltype='pred_celltype_l3', 
                                          report_name = 'report_test_model_scadam_635_AdamW_celltype_l3.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

Successfully added predicted celltype_l1 and cell type probabilities
Successfully added predicted celltype_l2 and cell type probabilities
Successfully added predicted celltype_l3 and cell type probabilities
Successfully saved report

Successfully saved report

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

Successfully added predicted celltype_l1 and cell type probabilities
Successfully added predicted celltype_l2 and cell type probabilities
Successfully added predicted celltype_l3 and cell type probabilities
Successfully saved report

Successfully saved report

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

S

In [16]:
scparadise.scadam.tune(adata_balanced, 
                       path = '/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/models/',
                       celltype_l1 = 'celltype_l1',
                       celltype_l2 = 'celltype_l2',
                       celltype_l3 = 'celltype_l3',
                       model_name = 'model_tuning_final',
                       eval_metric=['accuracy'])

Successfully saved genes names for training model

Successfully saved dictionary of dataset annotations

Accelerator: cuda



[I 2024-09-16 15:38:38,157] A new study created in RDB with name: model_tuning_final


Fold 1:

Early stopping occurred at epoch 84 with best_epoch = 74 and best_valid_accuracy = 0.96376

Fold 2:

Early stopping occurred at epoch 97 with best_epoch = 87 and best_valid_accuracy = 0.96554

Fold 3:

Early stopping occurred at epoch 72 with best_epoch = 62 and best_valid_accuracy = 0.96589

Fold 4:

Early stopping occurred at epoch 84 with best_epoch = 74 and best_valid_accuracy = 0.96802


[I 2024-09-16 15:48:17,807] Trial 0 finished with value: 0.965800702811245 and parameters: {'n_d': 8, 'n_a': 8, 'n_steps': 3, 'n_shared': 2, 'cat_emb_dim': 1, 'n_independent': 1, 'gamma': 1.3, 'momentum': 0.02, 'lr': 0.01, 'mask_type': 'entmax', 'lambda_sparse': 0.001, 'patience': 10, 'max_epochs': 100, 'virtual_batch_size': 128, 'batch_size': 1024}. Best is trial 0 with value: 0.965800702811245.



Fold 1:

Early stopping occurred at epoch 21 with best_epoch = 6 and best_valid_accuracy = 0.42174

Fold 2:

Early stopping occurred at epoch 71 with best_epoch = 56 and best_valid_accuracy = 0.60417

Fold 3:

Early stopping occurred at epoch 71 with best_epoch = 56 and best_valid_accuracy = 0.5562

Fold 4:

Early stopping occurred at epoch 51 with best_epoch = 36 and best_valid_accuracy = 0.5429


...

[I 2024-09-19 12:25:27,530] Trial 98 pruned. 


Fold 1:
Stop training because you reached max_epochs = 95 with best_epoch = 89 and best_valid_accuracy = 0.97289

Fold 2:
Stop training because you reached max_epochs = 95 with best_epoch = 81 and best_valid_accuracy = 0.97638

Fold 3:
Stop training because you reached max_epochs = 95 with best_epoch = 79 and best_valid_accuracy = 0.97701

Fold 4:
Stop training because you reached max_epochs = 95 with best_epoch = 83 and best_valid_accuracy = 0.975


[I 2024-09-19 12:41:58,999] Trial 99 finished with value: 0.975320030120482 and parameters: {'n_d': 108, 'n_a': 116, 'n_steps': 10, 'n_shared': 1, 'cat_emb_dim': 5, 'n_independent': 8, 'gamma': 1.255725850280637, 'momentum': 0.3309144445780589, 'lr': 0.03542773573542414, 'mask_type': 'entmax', 'lambda_sparse': 0.046967699654958665, 'patience': 20, 'max_epochs': 95, 'virtual_batch_size': 512, 'batch_size': 2560}. Best is trial 68 with value: 0.9758659638554217.




Best hyperparameters: {'n_d': 96, 'n_a': 56, 'n_steps': 5, 'n_shared': 6, 'cat_emb_dim': 6, 'n_independent': 5, 'gamma': 1.5752599469151702, 'momentum': 0.19797242376497604, 'lr': 0.05423890090439394, 'mask_type': 'entmax', 'lambda_sparse': 0.00046531419124969537, 'patience': 20, 'max_epochs': 100, 'virtual_batch_size': 512, 'batch_size': 1536}


{'n_d': 96,
 'n_a': 56,
 'n_steps': 5,
 'n_shared': 6,
 'cat_emb_dim': 6,
 'n_independent': 5,
 'gamma': 1.5752599469151702,
 'momentum': 0.19797242376497604,
 'lr': 0.05423890090439394,
 'mask_type': 'entmax',
 'lambda_sparse': 0.00046531419124969537,
 'patience': 20,
 'max_epochs': 100,
 'virtual_batch_size': 512,
 'batch_size': 1536}

In [20]:
scparadise.scadam.train_tuned(adata_balanced,
                              path_tuned='/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/models/model_tuning_final/',
                              path='/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/models/',
                              model_name='model_tuned_final',
                              celltype_l1='celltype_l1',
                              celltype_l2='celltype_l2',
                              celltype_l3='celltype_l3',
                              eval_metric=['balanced_accuracy', 'logloss', 'accuracy'])

Successfully saved genes names for training model

Successfully saved dictionary of dataset annotations

Train dataset contains: 47808 cells, it is 90.0 % of input dataset
Test dataset contains: 5312 cells, it is 10.0 % of input dataset

Accelerator: cuda
Start training with following hyperparameters: {'n_d': 96, 'n_a': 56, 'n_steps': 5, 'n_shared': 6, 'cat_emb_dim': 6, 'n_independent': 5, 'gamma': 1.5752599469151702, 'momentum': 0.19797242376497604, 'optimizer_params': {'lr': 0.05423890090439394}, 'mask_type': 'entmax', 'lambda_sparse': 0.00046531419124969537, 'patience': 20, 'max_epochs': 100, 'virtual_batch_size': 512, 'batch_size': 1536, 'device_name': 'cuda'}

epoch 0  | loss: 2.38221 | train_balanced_accuracy: 0.42554 | train_logloss: 1.77445 | train_accuracy: 0.47896 | valid_balanced_accuracy: 0.43097 | valid_logloss: 1.7534  | valid_accuracy: 0.48168 |  0:00:03s
epoch 1  | loss: 0.94374 | train_balanced_accuracy: 0.67815 | train_logloss: 0.83062 | train_accuracy: 0.72456 | vali

In [24]:
for folder in lst_test:
    adata_test = adata[adata.obs['orig.ident'].isin([folder[0:4], folder[5:9]])].copy()
    # Predict annotation levels using pretrained scadam model
    adata_test = scparadise.scadam.predict(adata_test, 
                                           path_model = '/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/models/model_tuned_final/')
    # Create and save classification report of annotation levels
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l1', 
                                          pred_celltype='pred_celltype_l1', 
                                          report_name = 'report_test_model_scadam_tuned_celltype_l1.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l2', 
                                          pred_celltype='pred_celltype_l2', 
                                          report_name = 'report_test_model_scadam_tuned_celltype_l2.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'celltype_l3', 
                                          pred_celltype='pred_celltype_l3', 
                                          report_name = 'report_test_model_scadam_tuned_celltype_l3.csv',
                                          save_path = os.path.join('/mnt/c/Users/vadim/scRNA/scParadise/scAdam/PBMC/3p/scparadise/reports_small/', folder).replace("\\","/"),
                                          save_report=True)

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

Successfully added predicted celltype_l1 and cell type probabilities
Successfully added predicted celltype_l2 and cell type probabilities
Successfully added predicted celltype_l3 and cell type probabilities
Successfully saved report

Successfully saved report

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

Successfully added predicted celltype_l1 and cell type probabilities
Successfully added predicted celltype_l2 and cell type probabilities
Successfully added predicted celltype_l3 and cell type probabilities
Successfully saved report

Successfully saved report

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded dictionary of dataset annotations

Successfully loaded model

S

In [25]:
pip list

Package                       Version
----------------------------- --------------
absl-py                       2.1.0
adjustText                    1.2.0
aiohappyeyeballs              2.3.6
aiohttp                       3.10.3
aiosignal                     1.3.1
alabaster                     1.0.0
alembic                       1.13.2
anndata                       0.10.8
annoy                         1.17.3
anyio                         4.4.0
appdirs                       1.4.4
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
array_api_compat              1.8
arrow                         1.3.0
asttokens                     2.4.1
async-lru                     2.0.4
attrs                         24.2.0
Babel                         2.14.0
beautifulsoup4                4.12.3
bleach                        6.1.0
bokeh                         3.4.2
branca                        0.7.2
Brotli                        1.1.0
cached-property               1.5.2
cachetools